In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import boto3
import json
import uuid

In [2]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('ds-niaimages')

folders = set()

for obj in bucket.objects.all():
    prefix, delimiter, _ = obj.key.rpartition('/')
    if prefix:
      folders.add(prefix + '/')
list(folders)

['20230816_1049-1053_30m/']

In [28]:
client = boto3.client('s3')



response = client.list_objects_v2(
    Bucket='ds-niaimages',
    # Prefix='/', 
    # Delimiter='/'
    )

folders = []

for obj in bucket.objects.all():
    prefix, delimiter, _ = obj.key.rpartition('/')
    if prefix:
        folders.add(prefix + '/')


for obj in response['Contents']:
  # if obj['Key'].endswith('/'):
    folders.append(obj['Key'])

response

{'ResponseMetadata': {'RequestId': 'MX1XWK7B7067CAVF',
  'HostId': 'f933m9woMgGkDByQYefc6KP4jkK9FjMtveKOQc0DZ5SWi2HWjM0szIMGDX6/7T5RzC9mdvhPR60=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'f933m9woMgGkDByQYefc6KP4jkK9FjMtveKOQc0DZ5SWi2HWjM0szIMGDX6/7T5RzC9mdvhPR60=',
   'x-amz-request-id': 'MX1XWK7B7067CAVF',
   'date': 'Sun, 27 Aug 2023 01:15:14 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': '20230816_1049-1053_30m/DJI_20230816105153_0356_D.JPG',
   'LastModified': datetime.datetime(2023, 8, 27, 0, 37, 17, tzinfo=tzutc()),
   'ETag': '"f8bea9e8c3d64927201ba896c077a767"',
   'Size': 5275648,
   'StorageClass': 'STANDARD'},
  {'Key': '20230816_1049-1053_30m/DJI_20230816105153_0356_MS_G.TIF',
   'LastModified': datetime.datetime(2023, 8, 27, 0, 37, 19, tzinfo=tzutc()),
   'ETag': '"46ec25c591c491a1c73c02f89445

In [11]:
Target_year = 2023

pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.1f}'.format

User_Name = 'dsadmin'


AWS_folder = "./"

src_folder = os.path.join(AWS_folder,'Downloaded',User_Name, 'geojson')
out_folder = os.path.join(AWS_folder,'ToUpload',User_Name, 'geojson')
course_folder = os.path.join(AWS_folder,'ToUpload',User_Name, 'coursegeojson')
info_folder = os.path.join(AWS_folder,'ToUpload',User_Name)

AREA Definition File을 Load

In [4]:
files = glob.glob(os.path.join(info_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]
area_def = json.load(open(target_files[0], encoding="utf8"))

In [5]:
# Create Out folder if not exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

S3에 있는 GEOJSON화일을 Check ... Not load Yet

In [6]:

client = boto3.client('s3')

response = client.list_objects_v2(
    Bucket='dsmapdata',
    Prefix='samjigi'+'/geojson/')

list = [x['Key'] for x in response.get('Contents', [])]
list = [ x for x in list if ('MGC' in x) ]
list


['samjigi/geojson/MGC001.json', 'samjigi/geojson/MGC002.json']

base.json file이 모든 정보를 가지고 있으므로 이화일만 다운로드
그 후 내용 화일로 분리

In [7]:
s3 = boto3.client('s3')

for file_ in list:
  content_object = s3.get_object(Bucket = 'dsmapdata', Key = file_)
  file_content = content_object['Body'].read().decode('utf-8')
  json_content = json.loads(file_content)

  for json_ in json_content['features']:
    json_['properties'] = ({
            "Name": "Contour_",
            "Id":str(uuid.uuid4()),
            "Client": json_['properties']['Client'],
            "Course": json_['properties']['Course'],            
            "Hole": 0 if json_['properties']['Hole'] == '' else int(float(json_['properties']['Hole'])),
            "Type": json_['properties']['Type'],
            "TypeId": next(item for item in area_def if item['name'] == json_['properties']['Type'])['TypeId'],
            "Desc": "",
            "Valid":True,
            "By":"",
            "When":"",
            "Labels":[{ 'level1':"", 'level2':"", 'level3':"",'TurfType':""}],
            "LabelL1":"",
            "LabelL2":"",
            "TurfType":"",
            "Start": "",
            "End": "",
            "radius": json_['properties'].get('radius',0),    
            "center": json_['properties'].get('center',[]),    
          })

  #   json_['geometry']['coordinates'] = [json_['geometry']['coordinates'][0]+[json_['geometry']['coordinates'][0][0]]]
      
  file_to_save = os.path.join(out_folder ,file_.split('/')[2])
  print(file_to_save)
  with open(file_to_save, "w", encoding="utf-8") as file:
    file.write(json.dumps(json_content, ensure_ascii=False))

./ToUpload\dsadmin\geojson\MGC001.json
./ToUpload\dsadmin\geojson\MGC002.json


In [14]:
files = glob.glob(os.path.join(out_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]
target_files

['./ToUpload\\dsadmin\\geojson\\MGC001.json',
 './ToUpload\\dsadmin\\geojson\\MGC002.json']

In [15]:
os.path.splitext(os.path.basename(target_files[0]))[0]

'MGC001'

각 화일을 base.json으로 통합, 그후 가가 화일과 통합화일을 Upload

In [16]:
Total_json = {}

s3 = boto3.client('s3')
for _file in target_files:
  f_ = os.path.splitext(os.path.basename(_file))[0]
  json_content_ = json.load(open(_file, encoding="utf8"))
  Total_json[f_] = json_content_
  s3.put_object(Body=json.dumps(json_content_).encode('utf-8'),Bucket='dsmapdata',Key=User_Name+'/geojson/{}.json'.format(f_))
  
# s3.put_object(
#   Body=json.dumps(Total_json).encode('utf-8'),
#   Bucket='dsmapdata',
#   Key=User_Name + '/base.json'
#   )

In [12]:
files = glob.glob(os.path.join(course_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]
target_files

['./ToUpload\\dsadmin\\coursegeojson\\MGC001.json',
 './ToUpload\\dsadmin\\coursegeojson\\MGC002.json']

In [13]:
Total_json = {}

s3 = boto3.client('s3')
for _file in target_files:
  f_ = os.path.splitext(os.path.basename(_file))[0]
  json_content_ = json.load(open(_file, encoding="utf8"))
  Total_json[f_] = json_content_
  s3.put_object(Body=json.dumps(json_content_).encode('utf-8'),Bucket='dsmapdata',Key=User_Name+'/coursegeojson/{}.json'.format(f_))
  
# s3.put_object(
#   Body=json.dumps(Total_json).encode('utf-8'),
#   Bucket='dsmapdata',
#   Key=User_Name + '/base.json'
#   )